In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from cmath import nan
import pywt
import torch
import pywt
from pandarallel import pandarallel
import tqdm
from functools import partial
import glob
import json
from pathlib import Path
# pandarallel.initialize(progress_bar=True)

/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Label attack data

In [3]:
data_folder = '../../Data/road/attacks/'
meta_file = data_folder + '/capture_metadata.json'
meta_data = json.load(open(meta_file))
meta_df = pd.DataFrame.from_dict(meta_data, orient='index')
meta_df

,description,elapsed_sec,injection_data_str,injection_id,injection_interval,modified,on_dyno
accelerator_attack_drive_1,start from park; drive; car accelerates until ...,86.462910,None,None,None,False,True
accelerator_attack_drive_2,start from park; drive; car accelerates until ...,72.868275,None,None,None,False,True
accelerator_attack_reverse_1,start from park; reverse; car accelerates unti...,86.134490,None,None,None,False,True
accelerator_attack_reverse_2,start from park; reverse; car accelerates unti...,105.438372,None,None,None,False,True
correlated_signal_attack_1,start from driving; accelerate; start injectin...,33.101852,595945450000FFFF,0x6e0,"[9.191851, 30.050109]",False,True
correlated_signal_attack_1_masquerade,start from driving; accelerate; start injectin...,33.101852,595945450000FFFF,0x6e0,"[9.191851, 30.050109]",True,True
correlated_signal_attack_2,start from driving; accelerate; start injectin...,28.226893,595945450000FFFF,0x6e0,"[6.830477, 28.225908]",False,True
correlated_signal_attack_2_masquerade,start from driving; accelerate; start injectin...,28.226893,595945450000FFFF,0x6e0,"[6.830477, 28.225908]",True,True
correlated_signal_attack_3,start from driving; accelerate; start injectin...,16.963905,595945450000FFFF,0x6e0,"[4.318482, 16.95706]",False,True
correlated_signal_attack_3_masquerade,start from driving; accelerate; start injectin...,16.963905,595945450000FFFF,0x6e0,"[4.318482, 16.95706]",True,True


In [48]:

attack_name = 'reverse_light_on_attack_1'
input_file = Path(data_folder) / f'{attack_name}.log'
df = pd.read_csv(input_file, sep=" ", header=None, names=['timestamp', 'interface', 'frame'])
df['timestamp'] = df['timestamp'].apply(lambda x: float(x[1:-1]))
df['timestamp'] = df['timestamp'] - df.iloc[0]['timestamp']
df['id'] = df['frame'].apply(lambda x: x.split('#')[0])
df['data'] = df['frame'].apply(lambda x: x.split('#')[1])
df['label'] = False
df = df.drop(['frame', 'interface'], axis=1)
df

,timestamp,id,data,label
0,0.000000e+00,FFF,0000000000000000,False
1,9.536743e-07,498,84002000FB800001,False
2,1.907349e-06,3C1,7B551FBC6C38292E,False
3,3.099442e-06,207,04046048291FB518,False
4,1.406670e-05,4C9,01F915820D02F64B,False
...,...,...,...,...
133232,5.484920e+01,162,00080003EA11F4CE,False
133233,5.484920e+01,0A7,0010FA24E12B00A0,False
133234,5.485020e+01,00E,2052960208097552,False
133235,5.485021e+01,26E,0106408289B60A04,False


In [58]:
start_injected, end_injected = meta_df.loc[attack_name].injection_interval
print(f'The attack {attack_name} starts from {start_injected} to {end_injected}')
injection_id = meta_df.loc[attack_name].injection_id
injection_data = meta_df.loc[attack_name].injection_data_str
injection_id = (injection_id[2:].upper()).zfill(3)
print(injection_id, injection_data)

The attack reverse_light_on_attack_1 starts from 18.929177 to 38.836015
0D0 XXXX0CXXXXXXXXXX


In [59]:
def find_change_byte(data_str):
    for idx, ch in enumerate(data_str):
        if ch != 'X': return idx
    return -1

In [66]:
idx = find_change_byte(injection_data)
if idx == -1:
    regex_pattern = injection_data
else:
    regex_pattern = "(.{" + str(idx) + "}" + injection_data[idx:idx+2] + ".{" + str(len(injection_data) - idx - 2) + "})"
regex_pattern

'(.{4}0C.{10})'

In [67]:
time_mask = ((df.timestamp >= start_injected) & (df.timestamp <= end_injected))
attack_mask = ((df.data.str.match(regex_pattern)) & (df.id == injection_id))
attack_indices = df[time_mask & attack_mask].index

In [68]:
df.loc[attack_indices, 'label'] = True
df

,timestamp,id,data,label
0,0.000000e+00,FFF,0000000000000000,False
1,9.536743e-07,498,84002000FB800001,False
2,1.907349e-06,3C1,7B551FBC6C38292E,False
3,3.099442e-06,207,04046048291FB518,False
4,1.406670e-05,4C9,01F915820D02F64B,False
...,...,...,...,...
133232,5.484920e+01,162,00080003EA11F4CE,False
133233,5.484920e+01,0A7,0010FA24E12B00A0,False
133234,5.485020e+01,00E,2052960208097552,False
133235,5.485021e+01,26E,0106408289B60A04,False


In [69]:
df.loc[attack_indices]

,timestamp,id,data,label
45277,18.929177,0D0,726D0C60DC021000,True
45296,18.936181,0D0,7A6D0C60DB021000,True
45323,18.947171,0D0,026D0C60E9021100,True
45348,18.957196,0D0,0A6D0C60E8021100,True
45369,18.967155,0D0,126D0C60E6021200,True
...,...,...,...,...
94822,38.796127,0D0,0A6D0C60CD022C00,True
94847,38.807477,0D0,126D0C60CB022D00,True
94880,38.818644,0D0,1A6D0C60CA022D00,True
94910,38.829072,0D0,226D0C60CA022C00,True
